In [1]:

import gzip
import csv
import pandas as pd
import numpy as np
import re
import glob
from collections import defaultdict
from sklearn.preprocessing import MinMaxScaler,StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from random import random
from math import exp
from collections import defaultdict

In [5]:
import seaborn as sn
import matplotlib.pyplot as plt
# from keras.wrappers.scikit_learn import KerasClassifier
# from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import explained_variance_score
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input
from sklearn import preprocessing
from tensorflow.keras.optimizers import SGD
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.feature_selection import VarianceThreshold
from pandas.io.formats.format import CategoricalFormatter
from sklearn.metrics import confusion_matrix
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
path='/content/drive/My Drive/Courses/Advanced topic/project/Dataset/'

In [8]:
df_pdi=pd.read_csv(path+'PDI/Copy of PDI.csv',sep=',')
df_pdi

,TF,Target
0,AHR,CYP1A1
1,AHR,CYP1A2
2,AHR,CYP1B1
3,AHR,FOS
4,AHR,MYC
...,...,...
32438,ZNF766,MARF1
32439,ZNF83,CUBN
32440,ZNF83,PCBP1
32441,ZNF83,SND1


In [9]:
# df=pd.read_csv("/content/drive/My Drive/Courses/Advanced topic/project/Dataset/Gene Expression/geneexpression_with_gene_name.csv", index_col=0)
# df=df.drop(columns=df.columns[df.eq(0).mean()>0.35])
df=pd.read_csv("/content/drive/My Drive/Courses/Advanced topic/project/Dataset/gene_expression_with_common_samples_with_dna_m.csv",index_col=0)

In [10]:
import keras.backend as K
class CustomConnected(Dense):

    def __init__(self,units,connections,**kwargs):

        #this is matrix A
        # self.set_weights= connections
        self.connections = connections                        

        #initalize the original Dense with all the usual arguments   
        super(CustomConnected,self).__init__(units,**kwargs)  


    def call(self, inputs):
      output = K.dot(inputs, self.kernel * self.connections)
      if self.use_bias:
          output = K.bias_add(output, self.bias)
      if self.activation is not None:
          output = self.activation(output)
      return output

In [11]:
run=False
if(run):
  gene_names= list(df.columns)
  gene_names= gene_names[:-1]
  df_pdi = df_pdi[df_pdi.TF.isin(gene_names)]
  df_pdi = df_pdi[df_pdi.Target.isin(gene_names)]
  d=defaultdict(lambda: defaultdict(int))
  for row in df_pdi.iterrows():
    t=row[1][0]
    g=row[1][1]
    d[g][t]=1
    # d[t][g]=1
  tfs=df_pdi['TF']
  tfs=np.unique(tfs)
  print(len(tfs))
  for g in gene_names:
    if(g not in d.keys()):
      print(g)
      for t in tfs:
        d[g][t]=0
        # d[t][g]=0

In [12]:
df_trans=df
df_trans=df_trans[df_trans['regions']!='NC']

In [13]:
df_pdi=pd.read_csv(path+'PDI/PDI_weights2.csv',index_col=0)
df_pdi= df_pdi.sort_values(by=['Unnamed: 0.1'])
index = df_pdi['Unnamed: 0.1']
df_pdi= df_pdi.drop(['Unnamed: 0.1'],axis=1)
df_pdi.index =index
df_pdi

,ATF3,EGR1,HNF4A,BHLHE40,CEBPA,KLF6,ATF4,JUN,KLF4,FOS,...,JUND,NFE2L2,JUNB,YBX1,KLF3,NR4A1,ELF1,XBP1,EHF,HES1
Unnamed: 0.1,,,,,,,,,,,,,,,,,,,,,
AAMP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ABCE1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ABCF1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ABHD11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ABHD12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZNF593,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZNF706,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZNHIT1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
df_trans

,AAMP,ABCE1,ABCF1,ABHD11,ABHD12,ABHD14B,ABHD2,ABI1,ABRACL,ACAA2,...,ZFR,ZMYND8,ZNF207,ZNF511,ZNF593,ZNF706,ZNHIT1,ZNRD1,ZYX,regions
samples,,,,,,,,,,,,,,,,,,,,,
CRC01_LN1_162,0.00,119.22,0.51,8.25,0.00,299.18,95.28,0.75,0.00,476.04,...,6.43,11.41,2.21,258.59,0.00,0.88,0.00,12.48,313.47,LN
CRC01_LN1_164,55.07,0.19,0.00,18.61,1.20,0.00,0.09,74.91,470.18,0.00,...,0.00,0.00,584.04,0.00,956.83,32.47,0.79,12.18,647.18,LN
CRC01_LN1_166,0.00,0.00,0.00,0.00,0.00,296.02,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,2.31,103.48,34.66,37.70,LN
CRC01_LN1_167,59.59,49.70,0.00,28.95,187.53,137.77,32.95,0.41,103.46,435.50,...,0.27,210.70,59.82,46.02,200.44,89.48,123.84,17.40,183.39,LN
CRC01_LN1_168,250.16,0.49,7.16,129.18,214.55,291.39,52.96,37.77,129.93,143.64,...,0.00,11.60,283.50,311.13,0.00,28.94,242.32,36.36,258.28,LN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CRC11_PT4_741,83.12,23.75,302.78,124.67,5.94,178.11,148.42,0.00,130.61,451.20,...,0.00,100.93,5.94,59.37,89.05,534.32,35.62,154.36,0.00,PT
CRC11_PT4_742,50.85,162.72,386.46,44.07,176.28,0.00,37.29,37.29,142.38,342.39,...,159.33,0.00,61.02,88.14,71.19,1033.96,396.63,64.41,47.46,PT
CRC11_PT4_746,126.04,0.00,210.07,0.00,84.03,0.00,0.00,126.04,168.05,168.05,...,168.05,168.05,0.00,0.00,0.00,126.04,126.04,168.05,42.01,PT


In [15]:
if(np.all(df_pdi.index == df_trans.columns[:-1])):
  print("same")

same


In [16]:
print(df_pdi.sum(axis=0).values)

[ 15. 137. 120.  13.  90.  19.  30.  31.  14. 103. 160.   9.   1.  19.
  26.   3.   8.   5.   9.  12.  18.  12.   2.   3.   5.  11.   2.   6.
   1.]


In [17]:
weights_pdi=[]
for i,cols in enumerate(df_pdi.columns):
  weights_pdi.append(df_pdi[cols].values)

In [18]:
extra_connections=False
if(extra_connections):
  for i in range(100):
    weights_pdi.append(np.ones(1944))

In [19]:
print(len(weights_pdi[0]))
weights_pdi=np.array(weights_pdi)

1944


In [20]:
weights_pdi

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [21]:
def create_model_tuning(layer_one=50,  layer_two=10,  activation_fuction_one='relu',activation_fuction_two='relu',drop_out_p=0.4, learningrate=0.1):
  model = tf.keras.models.Sequential()
  model.add(tf.keras.Input(shape=(data.shape[1],)))
  model.add(tf.keras.layers.Dense(weights_pdi.shape[0],kernel_initializer='lecun_uniform', activation=activation_fuction_one))
  model.add(Dropout(drop_out_p))
  model.add(tf.keras.layers.Dense(layer_one,kernel_initializer='lecun_uniform', activation=activation_fuction_two))  
  # model.add(tf.keras.layers.Dense(layer_two, activation='sigmoid'))
  model.add(tf.keras.layers.Dense(4, activation='sigmoid'))
  model.layers[0].set_weights([weights_pdi.T, np.zeros(model.layers[0].get_weights()[1].shape)])
  optimizer = SGD(learning_rate=learningrate, momentum=0.03)
  model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy',metrics= 'accuracy')
  
  return model

In [22]:
RANDOME_STATE=2000
def tune_hyperparameter(X,y):
  reports=[]
  test_score=[]
  train_score=[]
  best_score=-1
  cf=[]
  best_model=  create_model_tuning(1,1,'relu','relu',0.4,0.1)
  Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=RANDOME_STATE,stratify=y)
  layer_one=[200]
  # layer_two= [2, 4, 6, 8, 10]
  l2=2
  activation_fuction_one = ['relu','sigmoid','tanh']
  activation_fuction_two = ['relu','sigmoid','tanh']
  drop_out_p = [0.1,0.2,0.3,0.4]
  learningrate = [0.1]
  lo,lt,afo,aft,dop,lrt = [],[],[],[],[],[]
  acc_test, f1_test= [],[]
  acc_train,f1_train = [],[]
  for l1 in layer_one:
    # for l2 in layer_two:
      for af1 in activation_fuction_one:
        for af2 in activation_fuction_two:
          for d in drop_out_p:
            for l in learningrate:
              model=create_model_tuning(l1,l2,af1,af2, d, l)
              lo.append(l1)
              lt.append(l2)
              afo.append(af1)
              print(af2)
              aft.append(af2)
              dop.append(d)
              lrt.append(l)    
              model.fit(Xtrain,ytrain,validation_data=(Xtest,ytest), epochs=50)
              y_pred=model.predict(Xtest)
              max_value = np.argmax(y_pred, axis=1)
              y_pred_train=model.predict(Xtrain)
              max_value_train = np.argmax(y_pred_train, axis=1)
              if(best_score<f1_score(ytest, max_value, average='macro')):
                best_score=f1_score(ytest, max_value, average='macro')
                best_model=model
              f1_test.append(f1_score(ytest, max_value, average='macro'))
              f1_train.append(f1_score(ytrain, max_value_train, average='macro'))
              acc_test.append(accuracy_score(ytest, max_value))
              acc_train.append(accuracy_score(ytrain, max_value_train))
              tuning_result= pd.DataFrame(list(zip(lo,lt,afo,aft,dop,lrt,f1_test,f1_train,acc_test,acc_train)),columns =['layer_one','layer_two','activation_one','activation_two','dropout','learning_rate','f1_val','f1_train','acc_val','acc_tests'])
              tuning_result.to_csv("/content/drive/My Drive/Courses/Advanced topic/project/Hyper-parameter tuning/hyperparameter_tuning_with_gene_Expression_pdi_extra_connections_200.csv")
  print(tuning_result)
 
  return best_model,tuning_result

In [23]:
le = preprocessing.LabelEncoder()
scaler = StandardScaler()
print(len(df_trans['regions']))
print(len(df_trans))
le.fit(df_trans['regions'])

data=df_trans.drop(['regions'],axis=1)
# data=df_trans.
data=data.iloc[:,:]
data=(np.array(data).astype(np.float32))
scaler.fit(data)
data=scaler.transform(data)
# y=le.transform(df_trans['regions'])

# one hot
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y = enc.fit(df_trans['regions'].values.reshape(-1, 1))
y= enc.transform(df_trans['regions'].values.reshape(-1, 1))

609
609


In [24]:
y

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]])

In [25]:
tuning = False
if(tuning):
  X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.5, random_state=RANDOME_STATE,stratify=y)
  best_model, results=tune_hyperparameter(X_train,y_train)
  y_pred = best_model.predict(X_test)
  max_value = np.argmax(y_pred, axis=1)
  f1_score(y_test, max_value, average='macro')
  confusion_matrix(y_test, max_value)
  best_model.summary()
  results.to_csv("/content/drive/My Drive/Courses/Advanced topic/project/Hyper-parameter tuning/hyperparameter_tuning_with_gene_Expression_pdi_extra_connections_200.csv")

In [26]:
def create_model():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.Input(shape=(data.shape[1],)))
  model.add(CustomConnected(weights_pdi.shape[0],tf.convert_to_tensor(weights_pdi.T, dtype=tf.float32),activation='sigmoid'))
  model.add(Dropout(0.3))
  model.add(tf.keras.layers.Dense(100,kernel_initializer='lecun_uniform', activation='relu'))
  # model.add(tf.keras.layers.Dense(weights_pdi.shape[0],kernel_initializer='lecun_uniform', activation='tanh'))
  # model.add(Dropout(0.4))
  # model.add(tf.keras.layers.Dense(100,kernel_initializer='lecun_uniform', activation='sigmoid'))
  # model.add(tf.keras.layers.Dense(100,kernel_initializer='lecun_uniform', activation='relu'))
  # model.add(tf.keras.layers.Dense(8, activation='relu'))
  model.add(tf.keras.layers.Dense(y.shape[1], activation='softmax'))
  optimizer = SGD(learning_rate=0.1, momentum=0.03)
  # model.layers[0].set_weights([weights_pdi.T, np.zeros(model.layers[0].get_weights()[1].shape)])
  model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics= 'accuracy')
  return model

In [27]:
def stratfiedk_fold_cross_validation(X,y,y2,n,r):
  skf = StratifiedKFold(n_splits=n, shuffle=True, random_state=r)
  best_score=-1
  best_model=create_model()
  for train, test in skf.split(X,y2):
    Xtrain, X_val =X[train], X[test]
    ytrain, y_val =y[train], y[test]
    model=create_model()
    model.fit(Xtrain,ytrain,validation_data=(X_val,y_val),epochs=50)
    y_pred=model.predict(X_val)
    # max_value = np.argmax(y_pred, axis=1)
    print(model.evaluate(X_val,y_val)[1])   
    if(model.evaluate(X_val,y_val)[1]>best_score):

      best_model=model
  
  return best_model

In [28]:
y.shape[1]

4

In [ ]:

# y2=le.transform(df_trans['regions'])
y2=le.transform(df_trans['regions'])
RANDOME_STATES =[100, 1000, 2000]
# X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=RANDOME_STATE,stratify=y)
# train_acc,test_acc,reports,best_model,confusion_matrixes=stratfiedk_fold_cross_validation(X_train,y_train,5)
f1_folds_scores=[]
for r in RANDOME_STATES:
  skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=r)
  for train, test in skf.split(data,y2):
    X_train, X_test =data[train], data[test]
    y_train, y_test =y[train], y[test]
    y2_train, y2_test =y2[train], y2[test]
    best_model=stratfiedk_fold_cross_validation(X_train,y_train,y2_train,5,r)
    y_pred=best_model.predict(X_test)
    # max_value = np.argmax(y_pred, axis=1)
    f1_folds_scores.append(classification_report(enc.inverse_transform(y_test), enc.inverse_transform(y_pred), output_dict=True ))

Epoch 1/50
13/13 [==============================] - 1s 25ms/step - loss: 1.3574 - accuracy: 0.3728 - val_loss: 1.2611 - val_accuracy: 0.4388
Epoch 2/50
13/13 [==============================] - 0s 7ms/step - loss: 1.3021 - accuracy: 0.4267 - val_loss: 1.2537 - val_accuracy: 0.4388
Epoch 3/50
13/13 [==============================] - 0s 7ms/step - loss: 1.2315 - accuracy: 0.4344 - val_loss: 1.3302 - val_accuracy: 0.4388
Epoch 4/50
13/13 [==============================] - 0s 8ms/step - loss: 1.2165 - accuracy: 0.4396 - val_loss: 1.1354 - val_accuracy: 0.5306
Epoch 5/50
13/13 [==============================] - 0s 7ms/step - loss: 1.1602 - accuracy: 0.4576 - val_loss: 1.1061 - val_accuracy: 0.5204
Epoch 6/50
13/13 [==============================] - 0s 7ms/step - loss: 1.1410 - accuracy: 0.4730 - val_loss: 1.0338 - val_accuracy: 0.4490
Epoch 7/50
13/13 [==============================] - 0s 9ms/step - loss: 1.0855 - accuracy: 0.4679 - val_loss: 1.0623 - val_accuracy: 0.3571
Epoch 8/50
13/13 [=

In [ ]:
f1_folds_scores[0]['macro avg']

In [ ]:
#  f1_folds_scores[0]['0']
classes=['0','1','2','3']
precisions, recalls, f1_scores,supports=[],[],[],[]
accurcies,m_precision,m_recall,m_f1_score,w_precision,w_recall,w_f1_score=[],[],[],[],[],[],[]
for l in range(0,len(f1_folds_scores)):
  print(f1_folds_scores[l])
  # for c in classes:
  #   precisions.append(f1_folds_scores[l][c]['precision'])
  #   recalls.append(f1_folds_scores[l][c]['recall'])
  #   f1_scores.append(f1_folds_scores[l][c]['f1-score'])
  #   supports.append(f1_folds_scores[l][c]['support'])
  accurcies.append(f1_folds_scores[l]['accuracy'])
  m_precision.append(f1_folds_scores[l]['macro avg']['precision'])
  m_recall.append(f1_folds_scores[l]['macro avg']['recall'])
  m_f1_score.append(f1_folds_scores[l]['macro avg']['f1-score'])
  w_precision.append(f1_folds_scores[l]['weighted avg']['precision'])
  w_recall.append(f1_folds_scores[l]['weighted avg']['recall'])
  w_f1_score.append(f1_folds_scores[l]['weighted avg']['f1-score'])


In [ ]:
results_classes= pd.DataFrame(list(zip(precisions,recalls,f1_scores,supports)),columns =['precision','recall','f1-score','support'])
results_classes

In [ ]:
overall_results= pd.DataFrame(list(zip(accurcies,m_precision,m_recall,m_f1_score,w_precision,w_recall,w_f1_score)),columns =['accuracy','macro_precision','macro_recall','macro_f1','weighted_precision','weighted_recall','weighted_f1_score'])
overall_results

In [ ]:
save=False
if(save):
  results_classes.to_csv("/content/drive/My Drive/Courses/Advanced topic/project/CV_results/gene_experession_PDI_common_samples_CV_gene_Expression_pdi_extra_connections.csv")
  overall_results.to_csv("/content/drive/My Drive/Courses/Advanced topic/project/CV_results/gene_experession_PDI_common_samples_CV_gene_Expression_pdi_extra_connections.csv")

In [ ]:
overall_results.describe()